# Stock Market Question Answering Agent

- Answer questions about individual stocks using (mostly) OpenBB functions
- Uses the OpenAI tools functionality and the [wrappers provided by OpenBB](https://docs.openbb.co/platform/user_guides/llm_mode)
- Define a set of tool functions
- When prompting, give OpenAI the set of tools and their descriptions in the prompt and also in the structured JSON format
- If OpenAI needs to run a function to respond to the prompt, it will provide the function signature and ask you to provide the return values before continuing
- this needs very little custom code and leverages the openbb tools but doesn't currently work as well. would possibly convert to langchain and debug the full prompt.


In [1]:
import os
import dotenv
import warnings
from datetime import datetime, timedelta

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import requests
import json

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI

from magentic import prompt_chain, FunctionCall, OpenaiChatModel

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

# class to make a tool and utility functions
from BB_agent_tool import BB_agent_tool, get_response, agent_query

import pdb

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'openai         {openai.__version__}')


pandas         2.2.2
obb            4.2.1
openai         1.30.3


# Connect to OpenBB

In [3]:
obb


OpenBB Platform v4.2.1

Utilities:
    /account
    /user
    /system
    /coverage

Routers:
    /commodity
    /crypto
    /currency
    /derivatives
    /econometrics
    /economy
    /equity
    /etf
    /fixedincome
    /index
    /news
    /quantitative
    /regulators
    /technical

Extensions:
    - commodity@1.1.1
    - crypto@1.2.1
    - currency@1.2.1
    - derivatives@1.2.1
    - econometrics@1.2.1
    - economy@1.2.1
    - equity@1.2.1
    - etf@1.2.1
    - fixedincome@1.2.1
    - index@1.2.1
    - news@1.2.1
    - quantitative@1.2.1
    - regulators@1.2.1
    - technical@1.2.1

    - alpha_vantage@1.2.1
    - benzinga@1.2.1
    - biztoc@1.2.1
    - cboe@1.2.1
    - ecb@1.2.1
    - econdb@1.1.1
    - federal_reserve@1.2.1
    - finra@1.2.1
    - finviz@1.1.1
    - fmp@1.2.1
    - fred@1.2.1
    - government_us@1.2.1
    - intrinio@1.2.1
    - nasdaq@1.2.1
    - oecd@1.2.1
    - polygon@1.2.1
    - sec@1.2.1
    - seeking_alpha@1.2.1
    - stockgrid@1.2.1
    - tiingo@1.2.

In [4]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)


In [5]:
obb.user


UserSettings

id: 0665a898-bc50-7539-8000-26e72b18fdb1
profile: {'hub_session': {'username': 'drucev', 'email': 'drucev@hotmail.com', 'primary_usage': 'personal', 'user_uuid': 'c866b4d2-c09b-4b13-abb7-a93f1ac3c2b7', 'token_type': 'bearer', 'access_token': SecretStr('**********')}}
credentials: {'benzinga_api_key': None, 'polygon_api_key': SecretStr('**********'), 'tradingeconomics_api_key': None, 'intrinio_api_key': None, 'econdb_api_key': None, 'fred_api_key': SecretStr('**********'), 'fmp_api_key': SecretStr('**********'), 'tiingo_token': SecretStr('**********')}
preferences: {'cache_directory': '/Users/drucev/OpenBBUserData/cache', 'chart_style': 'dark', 'data_directory': '/Users/drucev/OpenBBUserData', 'export_directory': '/Users/drucev/OpenBBUserData/exports', 'metadata': True, 'output_type': 'OBBject', 'plot_enable_pywry': True, 'plot_open_export': False, 'plot_pywry_height': 762, 'plot_pywry_width': 1400, 'request_timeout': 60, 'show_warnings': False, 'table_style': 'dark', 'use

In [6]:
obb.account


/account
    login
    logout
    save
    refresh
    

In [7]:
# Change a credential - only need once, gets stored in openbb cloud
# Change a credential - only need once, gets stored in openbb cloud
# For this demo 

# obb.user.credentials.polygon_api_key = os.environ['POLYGON_API_KEY']
# obb.user.credentials.alpha_vantage_api_key = os.environ['ALPHAVANTAGE_API_KEY']
# obb.user.credentials.fred_api_key = os.environ['FRED_API_KEY']
# obb.user.credentials.tiingo_token = os.environ['TIINGO_API_KEY']
# obb.user.credentials.fmp_api_key = os.environ['FMP_API_KEY']
# obb.user.credentials.biztoc_api_key = os.environ['BIZTOC_API_KEY']

# Save account changes to the Hub
# obb.account.save()

# Refresh account with latest changes since login
# obb.account.refresh()

# Logout
# obb.account.logout()


In [8]:
obb.equity.search("Merck", provider="sec").to_df().head(3)


,symbol,name,cik
0,MRK,"Merck & Co., Inc.",310158


In [31]:
symbol = "MRK"
company = "Merck"


In [10]:
obj = obb.equity.price.quote(symbol)


In [11]:
obj


OBBject

id: 0665a899-b797-76ee-8000-915f6fe4cdef
results: [{'symbol': 'MRK', 'asset_type': None, 'name': 'Merck & Co., Inc.', 'excha...
provider: fmp
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'fmp'}, 'standa...

In [12]:
obj.results


[FMPEquityQuoteData(symbol=MRK, asset_type=None, name=Merck & Co., Inc., exchange=NYSE, bid=None, bid_size=None, bid_exchange=None, ask=None, ask_size=None, ask_exchange=None, quote_conditions=None, quote_indicators=None, sales_conditions=None, sequence_number=None, market_center=None, participant_timestamp=None, trf_timestamp=None, sip_timestamp=None, last_price=125.54, last_tick=None, last_size=None, last_timestamp=2024-05-31 20:00:02+00:00, open=125.13, high=126.61, low=124.55, close=None, volume=17763542, exchange_volume=None, prev_close=124.53, change=1.01, change_percent=0.008111, year_high=133.1, year_low=99.14, price_avg50=128.1684, price_avg200=115.7113, avg_volume=8372884, market_cap=317968967400.0, shares_outstanding=2532810000, eps=0.9, pe=139.49, earnings_announcement=2024-07-30 12:30:00+00:00)]

In [13]:
obj.json()

'{"id":"0665a899-b797-76ee-8000-915f6fe4cdef","results":[{"symbol":"MRK","asset_type":null,"name":"Merck & Co., Inc.","exchange":"NYSE","bid":null,"bid_size":null,"bid_exchange":null,"ask":null,"ask_size":null,"ask_exchange":null,"quote_conditions":null,"quote_indicators":null,"sales_conditions":null,"sequence_number":null,"market_center":null,"participant_timestamp":null,"trf_timestamp":null,"sip_timestamp":null,"last_price":125.54,"last_tick":null,"last_size":null,"last_timestamp":"2024-05-31T20:00:02Z","open":125.13,"high":126.61,"low":124.55,"close":null,"volume":17763542,"exchange_volume":null,"prev_close":124.53,"change":1.01,"change_percent":0.008111,"year_high":133.1,"year_low":99.14,"price_avg50":128.1684,"price_avg200":115.7113,"avg_volume":8372884,"market_cap":317968967400.0,"shares_outstanding":2532810000,"eps":0.9,"pe":139.49,"earnings_announcement":"2024-07-30T12:30:00Z"}],"provider":"fmp","warnings":null,"chart":null,"extra":{"metadata":{"arguments":{"provider_choices":{

In [14]:
# use REST API on server running locally
# uvicorn openbb_core.api.rest_api:app --host 0.0.0.0 --port 8000 --reload
# REST API documentation - http://127.0.0.1:8000/docs
# openapi.json : http://127.0.0.1:8000/openapi.json

# not turning on authentication
# msg = "some_user:some_pass"
# msg_bytes = msg.encode('ascii')
# base64_bytes = base64.b64encode(msg_bytes)
# base64_msg = base64_bytes.decode('ascii')

url = f"http://127.0.0.1:8000/api/v1/equity/price/quote?provider=yfinance&symbol={symbol}"
# headers = {"accept": "application/json", "Authorization": f"Basic {base64_msg}"}
headers = {"accept": "application/json"}

response = requests.get(url=url, headers=headers)

response.json()


{'results': [{'symbol': 'MRK',
   'asset_type': 'EQUITY',
   'name': 'Merck & Co., Inc.',
   'exchange': 'NYQ',
   'bid': 125.58,
   'bid_size': 1000,
   'ask': 125.65,
   'ask_size': 1800,
   'last_price': 125.54,
   'open': 125.13,
   'high': 126.61,
   'low': 124.55,
   'volume': 17886698,
   'prev_close': 124.53,
   'year_high': 133.1,
   'year_low': 99.14,
   'ma_50d': 128.2022,
   'ma_200d': 115.7957,
   'volume_average': 8565420.0,
   'volume_average_10d': 8656260.0,
   'currency': 'USD'}],
 'provider': 'yfinance',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {'source': 'iex'}},
   'duration': 574438458,
   'route': '/equity/price/quote',
   'timestamp': '2024-05-31T22:38:27.084921'}}}

In [15]:
data = obb.equity.price.historical(symbol, provider="polygon")
data.to_dataframe()


,open,high,low,close,volume,vwap,transactions
date,,,,,,,
2023-05-31,108.47,111.15,108.470,110.41,14563536.0,110.3172,91481
2023-06-01,110.66,111.59,109.880,110.93,6960063.0,110.9360,76972
2023-06-02,111.11,112.73,110.720,112.52,5792792.0,112.1935,74396
2023-06-05,113.00,114.48,112.580,113.11,5692920.0,113.2613,76453
2023-06-06,113.55,113.72,109.280,110.01,6960217.0,110.5323,95257
...,...,...,...,...,...,...,...
2024-05-23,130.90,131.92,130.310,131.09,5416660.0,131.1242,70862
2024-05-24,131.45,131.45,129.160,129.49,5569920.0,129.7614,74137
2024-05-28,128.84,129.08,125.740,126.09,10151595.0,126.3212,114225


In [16]:
# use the local rest server
data = []
symbol2="SPY"
url = f"http://127.0.0.1:8000/api/v1/equity/price/historical?provider=polygon&symbol={symbol2}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, timeout=3)

if response.status_code == 200:
  data = OBBject.model_validate(response.json())

data.to_df()


,close,high,low,open,transactions,volume,vwap
date,,,,,,,
2023-05-31,417.85,419.2200,416.220,418.28,665147,110782530.0,418.0147
2023-06-01,421.82,422.9200,416.790,418.09,653406,88758018.0,420.4660
2023-06-02,427.92,428.7400,423.950,424.50,678309,91426171.0,426.8396
2023-06-05,427.10,429.6700,426.370,428.28,551795,70168016.0,427.9902
2023-06-06,428.03,428.5772,425.990,426.67,491099,63706675.0,427.5845
...,...,...,...,...,...,...,...
2024-05-23,525.96,533.0700,524.720,532.96,440792,56651841.0,528.0228
2024-05-24,529.44,530.2700,526.881,527.85,328927,41239368.0,528.9137
2024-05-28,529.81,530.5100,527.110,530.27,343539,36263860.0,529.3202


In [17]:
results = obb.equity.search(query='marvell', is_symbol=False, provider='sec', use_cache=True)
[(r.symbol, r.name) for r in results.results]


[('MRVL', 'Marvell Technology, Inc.')]

In [18]:
# multiple symbols
quotes = obb.equity.price.quote("td,schw,jpm,ms", provider="fmp")
quotes.to_df()


,symbol,name,exchange,last_price,last_timestamp,open,high,low,volume,prev_close,...,year_high,year_low,price_avg50,price_avg200,avg_volume,market_cap,shares_outstanding,eps,pe,earnings_announcement
0,TD,The Toronto-Dominion Bank,NYSE,55.92,2024-05-31 20:00:02+00:00,55.16,55.945,55.01,2353907,54.96,...,66.15,54.12,57.7658,59.70505,2975680,9.838006e+10,1759300000,4.37,12.80,2024-05-23 12:30:00+00:00
1,SCHW,The Charles Schwab Corporation,NYSE,73.28,2024-05-31 20:00:02+00:00,71.76,73.370,71.58,8562099,71.88,...,79.49,48.66,73.7096,63.79165,7423431,1.302391e+11,1777280000,2.39,30.66,2024-07-16 12:30:00+00:00
2,JPM,JPMorgan Chase & Co.,NYSE,202.63,2024-05-31 20:01:02+00:00,199.30,203.300,198.35,6952024,199.33,...,205.88,135.19,194.9104,168.86615,9086187,5.818865e+11,2871670000,16.56,12.24,2024-07-12 12:30:00+00:00
3,MS,Morgan Stanley,NYSE,97.84,2024-05-31 20:03:37+00:00,97.33,97.960,96.40,7147518,97.23,...,103.25,69.42,94.2054,86.38140,7677123,1.590057e+11,1625160000,5.50,17.79,2024-07-16 12:30:00+00:00


In [19]:
# multiple providers

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance(symbol, provider="yfinance", limit=3)
  .to_df().get("total_assets")
)

df["fmp"] = (
  obb.equity.fundamental.balance(symbol, provider="fmp", limit=3)
  .to_df().get("total_assets")
)

df["polygon"] = (
  obb.equity.fundamental.balance(symbol, provider="polygon", limit=3)
  .to_df().get("total_assets")
)

df

,yfinance,fmp,polygon
0,1.066750e+11,1.066750e+11,1.066750e+11
1,1.091600e+11,1.091600e+11,1.091600e+11
2,1.056940e+11,1.056940e+11,1.056940e+11
3,9.158800e+10,NaN,NaN


In [20]:
obb.news.company(symbol, provider='polygon', limit=5).to_df()


,title,text,images,url,symbols,source,id,amp_url,publisher
date,,,,,,,,,
2024-05-27 05:45:30+00:00,Unloved Gilead Due For Reversal On Flight-To-S...,Numerous drugs under development are projected...,[{'url': 'https://static.seekingalpha.com/cdn/...,https://seekingalpha.com/article/4695781-unlov...,"ABBV,AMGN,BIIB,BMY,BMYMP,BNTX,GLAXF,GSK,LLY,MR...",Paul Franke,KwL4UVXZJPnuntp2y2lmCEIYOd6pebYypyY9SHc06wI,None,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-28 14:38:00+00:00,Moderna (MRNA) Stock Outperformed Industry YTD...,Shares of Moderna (MRNA) are being driven by e...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2280158/moder...,"MRK,MRNA,HRTX,ARQT",Zacks Equity Research,Gw9nyWcRUQ6004OVcXK4bXv8jHJhiiNTjm-r4P8L534,https://www.zacks.com/amp/stock/news/2280158/m...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-30 16:24:00+00:00,Merck (MRK) to Buy Private Ophthalmology Compa...,Merck's (MRK) acquisition of EyeBio is set to ...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2281600/merck...,"JNJ,MRK,HRTX,BVS",Zacks Equity Research,cIh25UMm8bcOJrtzNcggBOMCUQ8LEimkKDSDVMkPgDY,https://www.zacks.com/amp/stock/news/2281600/m...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-31 10:20:06+00:00,Should Schwab U.S. Dividend Equity ETF (SCHD) ...,Style Box ETF report for SCHD,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2281786/shoul...,"SCHD,MRK,AVGO,ABBV,VTV,IWD",Zacks Equity Research,j-2nXYnN8cFFdV8CoQDymZAcMSZVTGmJU_ERMUXCJYI,https://www.zacks.com/amp/stock/news/2281786/s...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-31 13:33:00+00:00,"Pharma Stock Roundup: JNJ, MRK M&A Deals, SNY,...",J&J (JNJ) and Merck (MRK) announce the acquisi...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2281925/pharm...,"SNY,AZN,RHHBY,JNJ,MRK",Kinjel Shah,3zBaCt6YWLR_8kX9IMtbpk1U1kbwonXxgvcRnPqwNI4,https://www.zacks.com/amp/stock/news/2281925/p...,{'favicon_url': 'https://s3.polygon.io/public/...


# Prompt OpenAI 

In [8]:
MODEL = "gpt-4o"

# MAX_INPUT_TOKENS = 65536
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0

client = OpenAI()


In [22]:
messages = [{"role": "user", "content": "what is the airspeed velocity of an unladen swallow?"}]
response = get_response(client, messages, tools=[])
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\\$")

print(response_str)

The airspeed velocity of an unladen European Swallow (Hirundo rustica) has been estimated to be around 11 meters per second, or approximately 24 miles per hour. This figure is based on various studies of their flight mechanics and speed.

If this question is a nod to the classic line from the movie "Monty Python and the Holy Grail," then you've tapped into a humorous and frequently quoted piece of pop culture!


# Question Answering Agent

In [23]:
# make a custom tool

def get_10k_item1_from_symbol(symbol):
    """
    Get item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    # always return a list of dicts
    return [{'item1': item1_text}]

fn_metadata = {
    "name": "get_10k_item1_from_symbol",
    "description": "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    "openapi_path" : None,
    "callable": get_10k_item1_from_symbol,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The symbol to get the 10-K item 1 for"
            }
        },
    "default_parameters": {},
    "example_parameter_values": [{
        "symbol": "MSFT",
    }],
}

tool = BB_agent_tool(**fn_metadata)
tool_response = tool(symbol="MSFT")
len(json.loads(tool_response)[0]['item1'])


143226

In [24]:
user_message = "Please summarize item 1 from the latest MSFT annual report"
agent_query(client, user_message, verbose=True)



Role: You are an AI stock market assistant tasked with providing investors
with up-to-date, detailed information on individual stocks.

Objective: Assist data-driven stock market investors by giving accurate,
complete, but concise information relevant to their questions about individual
stocks.

Capabilities: You are given a number of tools as functions. Use as many tools
as needed to ensure all information provided is timely, accurate, concise,
relevant, and responsive to the user's query.

Instructions:
1. Input validation. Determine if the input is asking about a specific company
or stock ticker. If not, respond in a friendly, positive, professional tone
that you don't have information to answer and suggest alternative services
or approaches.

2. Symbol extraction. If the query is valid, extract the company name or ticker
symbol from the question. If a company name is given, look up the ticker symbol
using a tool. If the ticker symbol is not found based on the company, try to
corre

### Summary of Item 1 from Microsoft's Latest 10-K Annual Report

#### **General Description:**
- **Business Overview:**
  - **Mission:** Microsoft is dedicated to empowering individuals and organizations globally to achieve more.
  - **Products and Services:** They offer a wide array of software, services, devices, and solutions, including cloud-based services, software development tools, business applications, gaming consoles, and more.
  - **Key Segments:** Their operations are divided into three main segments:
    1. **Productivity and Business Processes:** Includes Office, LinkedIn, and Dynamics.
    2. **Intelligent Cloud:** Consists of Azure, SQL Server, and Nuance, among others.
    3. **More Personal Computing:** Encompasses Windows, Devices (Surface, Xbox), and Search and Advertising.

#### **Strategic Initiatives:**
- **AI Integration:** Microsoft is heavily investing in artificial intelligence (AI) across all their services, aiming to innovate and expand their offerings with AI capabilities.
- **Cloud Services:** Microsoft Cloud is designed to help organizations unlock value from their digital investments while ensuring security and operational efficiency.
- **Gaming:** Microsoft's strategy includes acquiring new content creators and developing new gaming studios to expand Xbox Game Pass and other services.
- **Sustainability:** Committed to being a carbon negative, water positive, and zero waste company by 2030.

#### **Commitment to Innovation:**
- **Research and Development:** The company continues to focus on developing new technologies and integrating them into their services to remain competitive.
- **Product Development:** Internal teams focus on areas such as cloud infrastructure, productivity tools, security, and devices.

#### **Challenges and Competition:**
- **Market Dynamics:** Facing intense competition from various software and application vendors, including Apple, Google, Amazon, and others.
- **Regulatory Risks:** Microsoft is subject to global regulatory scrutiny and must navigate a complex legal landscape regarding data privacy, antitrust laws, and more.
- **Security and Trust:** Ensuring the security of their products and services, along with addressing user privacy and data management concerns, are paramount.

#### **Corporate Social Responsibility:**
- **Racial Equity:** Initiatives to address racial injustice and promote equity within the organization and beyond are in place, with specific targets set for 2025.
- **Digital Skills:** Programs like the Skills for Jobs initiative aim to address the global skills gap, particularly in cybersecurity and AI.

#### **Operational Structure:**
- **Global Workforce:** Employing roughly 221,000 people worldwide, with significant investment in employee development and wellbeing.
- **Market Presence:** Microsoft operates datacenters and regional offices globally to support its diverse customer base.

This summary highlights the extensive scope of Microsoft's operations, strategic priorities, and their commitment to innovation and corporate responsibility. It underscores the challenges and competitive pressures they face in the technology sector, as well as their resilience and proactive measures in addressing them.

# Add more OpenBB tools
Map OpenBB functions to OpenAI tools

In [5]:
# reimport obb after doing this.
obb.user.preferences.output_type="llm"
obb.system.python_settings.docstring_sections=['description', 'examples']
obb.system.python_settings.docstring_max_length=1000
openbb.build()


In [5]:
#note short description
print(obb.equity.price.quote.__doc__)

Get the latest quote for a given stock. Quote includes price, volume, and other data.


        Examples
        --------
        >>> from openbb import obb
        >>> obb.equity.price.quote(symbol='AAPL', provider='fmp')
        


In [28]:
import inspect

# List all members (including methods)
# members = inspect.getmembers(obb.equity.price.quote)
# for m in members:
#     print(m)
#     print("====\n")

# List only methods
# methods = inspect.getmembers(obb.equity.price.quote, predicate=inspect.ismethod)
# print(methods)

# List only attributes
attributes = inspect.getmembers(obb.equity.price.quote, predicate=lambda a: not(inspect.isroutine(a)))
for a in attributes:
    print(a[0])
    print(a[1])
    print("====\n")
    

__class__
<class 'method'>
====

__doc__
Get the latest quote for a given stock. Quote includes price, volume, and other data.


        Examples
        --------
        >>> from openbb import obb
        >>> obb.equity.price.quote(symbol='AAPL', provider='fmp')
        
====

__self__
/equity/price
    historical
    nbbo
    performance
    quote
    
====



In [10]:
@prompt_chain(
    "You are a helpful financial agent that can use function calling to retrieve data.\nUser Query: {query}",
    functions=[obb.equity.price.quote],
    model=OpenaiChatModel(model=MODEL)
)
def llm(query: str) -> FunctionCall | str:
    return None

r = llm(query="What is the current stock price of AAPL?")

display(Markdown(r))

The current stock price of Apple Inc. (AAPL) is $192.48.

In [32]:
# load the OpenAPI / swagger spec from
# http://127.0.0.1:8000/openapi.json
with open("openapi.json", 'r') as file:
    data = json.load(file)

str(data)[:2000]


"{'openapi': '3.1.0', 'info': {'title': 'OpenBB Platform API', 'description': 'This is the OpenBB Platform API.', 'termsOfService': 'http://example.com/terms/', 'contact': {'name': 'OpenBB Team', 'url': 'https://openbb.co/', 'email': 'hello@openbb.co'}, 'license': {'name': 'MIT', 'url': 'https://github.com/OpenBB-finance/OpenBBTerminal/blob/develop/LICENSE'}, 'version': '1'}, 'servers': [{'url': 'http://localhost:8000', 'description': 'Local OpenBB development server'}], 'paths': {'/api/v1/commodity/lbma_fixing': {'get': {'tags': ['commodity'], 'summary': 'Lbma Fixing', 'description': 'Daily LBMA Fixing Prices in USD/EUR/GBP.', 'operationId': 'commodity_lbma_fixing', 'parameters': [{'name': 'provider', 'in': 'query', 'required': False, 'schema': {'enum': ['nasdaq'], 'const': 'nasdaq', 'type': 'string', 'default': 'nasdaq', 'title': 'Provider'}}, {'name': 'asset', 'in': 'query', 'required': False, 'schema': {'enum': ['gold', 'silver'], 'type': 'string', 'description': 'The metal to get 

In [33]:
# list all the equity functions
for path_str, fn_json in data['paths'].items():
    if path_str.find('equity') != -1:
        print(path_str)


/api/v1/equity/calendar/ipo
/api/v1/equity/calendar/dividend
/api/v1/equity/calendar/splits
/api/v1/equity/calendar/earnings
/api/v1/equity/compare/peers
/api/v1/equity/compare/groups
/api/v1/equity/estimates/price_target
/api/v1/equity/estimates/historical
/api/v1/equity/estimates/consensus
/api/v1/equity/estimates/analyst_search
/api/v1/equity/estimates/forward_sales
/api/v1/equity/estimates/forward_eps
/api/v1/equity/darkpool/otc
/api/v1/equity/discovery/gainers
/api/v1/equity/discovery/losers
/api/v1/equity/discovery/active
/api/v1/equity/discovery/undervalued_large_caps
/api/v1/equity/discovery/undervalued_growth
/api/v1/equity/discovery/aggressive_small_caps
/api/v1/equity/discovery/growth_tech
/api/v1/equity/discovery/top_retail
/api/v1/equity/discovery/upcoming_release_days
/api/v1/equity/discovery/filings
/api/v1/equity/fundamental/multiples
/api/v1/equity/fundamental/balance
/api/v1/equity/fundamental/balance_growth
/api/v1/equity/fundamental/cash
/api/v1/equity/fundamental/r

In [34]:
data['paths']['/api/v1/equity/search']


{'get': {'tags': ['equity'],
  'summary': 'Search',
  'description': 'Search for stock symbol, CIK, LEI, or company name.',
  'operationId': 'equity_search',
  'parameters': [{'name': 'provider',
    'in': 'query',
    'required': True,
    'schema': {'enum': ['cboe', 'intrinio', 'nasdaq', 'sec', 'tmx', 'tradier'],
     'type': 'string',
     'title': 'Provider'}},
   {'name': 'query',
    'in': 'query',
    'required': False,
    'schema': {'type': 'string',
     'description': 'Search query.',
     'default': '',
     'title': 'Query'},
    'description': 'Search query.'},
   {'name': 'is_symbol',
    'in': 'query',
    'required': False,
    'schema': {'type': 'boolean',
     'description': 'Whether to search by ticker symbol.',
     'default': False,
     'title': 'Is Symbol'},
    'description': 'Whether to search by ticker symbol.'},
   {'name': 'use_cache',
    'in': 'query',
    'required': False,
    'schema': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
     'descriptio

In [12]:
obb.equity.search(query="Merck")

OBBject

id: 0665a8cb-e3ba-7cc4-8000-bfc551f67ede
results: [{'symbol': 'MRK', 'name': 'MERCK & CO INC COM', 'dpm_name': 'Wolverine Tr...
provider: cboe
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'cboe'}, 'stand...

In [26]:
# use partial to fix issues with default provider
from functools import partial
# Create a partial function with provider defaulting to 'sec'
obb_equity_search_sec = partial(obb.equity.search, provider='sec')
obb_equity_search_sec.__name__ = "obb_equity_search_sec"
# call the partial function with just the 'query' argument
result = obb_equity_search_sec(query="Merck")
result

OBBject

id: 0665a8e7-aefb-700b-8000-f20af2517f21
results: [{'symbol': 'MRK', 'name': 'Merck & Co., Inc.', 'cik': '310158'}]
provider: sec
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'sec'}, 'standa...

In [124]:
@prompt_chain(
    "You are a helpful financial agent that can use function calling to retrieve data.\nUser Query: {query}",
    functions=[obb_equity_search_sec,
               obb.equity.price.quote,
               obb.equity.profile, 
               obb.equity.fundamental.historical_splits, 
               obb.equity.fundamental.balance, 
               obb.equity.fundamental.income, 
               obb.equity.fundamental.cash, 
               obb.equity.fundamental.metrics,
               obb.equity.fundamental.ratios, 
               obb.equity.fundamental.multiples, 
               obb.equity.fundamental.dividends,
               obb.equity.fundamental.trailing_dividend_yield,
               obb.equity.price.performance,
               obb.etf.equity_exposure               
              ],
    model=OpenaiChatModel(model=MODEL)
)
def llm(query: str) -> FunctionCall | str:
    return None

r = llm(query="What is the stock symbol for Merck?")
display(Markdown(r))


The stock symbol for Merck & Co., Inc. is **MRK**.

In [91]:
r = llm(query="What is the stock symbol for Delta airlines?")
display(Markdown(r))


It seems there were no results for the stock symbol search for "Delta Airlines" using the SEC database. Let me try again without restricting the search to symbols only.

In [92]:
r = llm(query="What is the stock symbol for yakimee airlines?")
display(Markdown(r))


It appears that there were no results for "Yakimee Airlines" using the search query provided. This could mean that either the company does not exist or is not listed under this name. If you have any other details or another company to look up, please let me know!

In [93]:
r = llm(query=f"What is the last market quote for symbol {symbol}?")
display(Markdown(r.replace("$", "\$")))


The latest market quote for the symbol MRK (Merck & Co., Inc.) is as follows:

- **Last Price:** \$125.54
- **Bid Price:** \$125.56
- **Ask Price:** \$131.03
- **Open:** \$125.06
- **High:** \$126.61
- **Low:** \$124.55
- **Close:** \$125.54
- **Volume:** 18,150,300
- **Change:** \$1.01
- **Change Percent:** 0.80%
- **52 Week High:** \$133.10
- **52 Week Low:** \$99.14

In [94]:
r = llm(query=f"What is the last market quote for company {company}?")
display(Markdown(r.replace("$", "\$")))


The latest market quote for Merck & Co., Inc. (MRK) is as follows:

- **Last Price:** \$125.54
- **Change:** +\$1.01
- **Change Percent:** +0.80%
- **Open:** \$125.06
- **High:** \$126.61
- **Low:** \$124.55
- **Close:** \$125.54
- **Volume:** 18,150,300
- **Year High:** \$133.10
- **Year Low:** \$99.14

In [36]:
# nonexistent company
r = llm(query=f"What is the last market quote for Consolidated Agglomerators")
display(Markdown(r.replace("$", "\$")))


The last market quote for "Consolidated Agglomerators" (AGG) is as follows:

- **Last Price:** \$96.52
- **Open Price:** \$96.46
- **High Price:** \$96.59
- **Low Price:** \$96.41
- **Close Price:** \$96.52
- **Volume:** 8,762,935 shares

### Bid and Ask Information:
- **Bid Price:** \$96.26 (Size: 7)
- **Ask Price:** \$96.76 (Size: 8)

### Additional Data:
- **Year High:** \$99.70
- **Year Low:** \$91.58

If you need more detailed or specific information, feel free to ask!

In [87]:
r = llm(query=f"Can you provide a basic company profile of symbol {symbol}")

display(Markdown(r.replace("$", "\$")))


### Company Profile: Merck & Co., Inc. (Ticker: MRK)

**Basic Information:**
- **Name:** Merck & Co., Inc.
- **Symbol:** MRK
- **Stock Exchange:** NYSE
- **Sector:** Healthcare
- **Industry Category:** Drug Manufacturers - General
- **Headquarters:** Rahway, NJ, USA
- **Employees:** 72,000

**Business Summary:**
Merck & Co., Inc. is a health care company engaged in providing health solutions through its prescription medicines, vaccines, biologic therapies, animal health, and consumer care products. The company operates through the following segments:

- **Pharmaceutical:** Human health pharmaceutical and vaccine products.
- **Animal Health:** Development, manufacture, and marketing of animal health products such as pharmaceutical and vaccine products for disease prevention, treatment, and control in livestock and companion animals.
- **Other:** Sales for the non-reportable segments of healthcare services.

**Financial Information:**
- **Shares Outstanding:** 2.53 Billion
- **Shares Float:** 2.53 Billion
- **Short Interest:** 20.27 Million
- **Institutional Ownership:** 78.69%
- **Market Cap:** \$317.97 Billion
- **Index:** DJIA, S&P 500
- **Earnings Date:** April 25 BMO
- **Beta:** 0.41

Merck & Co., Inc. was founded in 1891. For more detailed financial data and other specific information, additional resources may be required.

In [42]:
# tool = BB_agent_tool(
#     name="get_equity_shorts_short_interest",
#     description="Given a stock symbol, get data on short volume and days to cover in JSON format.",
#     openapi_path='/api/v1/equity/shorts/short_interest',
#     parameters={
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     default_parameters={
#     },    
#     example_parameter_values=[{
#         "symbol": "NVDA",
#     }],
# )

# agent_query(client, f"Provide latest statistics on short interest for symbol {symbol}", verbose=False)


In [89]:
r = llm(query=f"Can you provide historical split information for symbol AAPL")
display(Markdown(r.replace("$", "\$")))


Here are the historical stock splits for Apple Inc. (AAPL):

1. **August 31, 2020**
   - Split ratio: 4-for-1

2. **June 09, 2014**
   - Split ratio: 7-for-1

3. **February 28, 2005**
   - Split ratio: 2-for-1

4. **June 21, 2000**
   - Split ratio: 2-for-1

5. **June 16, 1987**
   - Split ratio: 2-for-1

If you need any more information, feel free to ask!

In [95]:
r = llm(query=f"what are the latest total assets for symbol {symbol}")
display(Markdown(r.replace("$", "\$")))


The latest total assets for the symbol MRK (Merck & Co., Inc.) are \$106.675 billion, as of the fiscal year ending on December 31, 2023.

In [96]:
r = llm(query=f"what are the total assets for fiscal year ending December 31, 2022 for symbol {symbol}")
display(Markdown(r.replace("$", "\$")))


For fiscal year ending December 31, 2022, the total assets for Merck & Co., Inc. (symbol: MRK) are \$109.16 billion.

You can view the detailed report [here](https://www.sec.gov/Archives/edgar/data/310158/000162828023005061/mrk-20221231.htm).

In [97]:
r = llm(query=f"what was the latest cash flow from operations for symbol {symbol}")
display(Markdown(r.replace("$", "\$")))


The latest cash flow from operations for Merck & Co., Inc. (symbol: MRK) was \$13,006,000,000 for the fiscal year ending on December 31, 2023.

For more details, you can view the [SEC filing here](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [103]:
r = llm(query=f"what was the latest net income for symbol MRK")
display(Markdown(r.replace("$", "\$")))


OpenBBError: 
[Error] -> 2 validations error(s)
[Arg] 2 -> input: 1 -> Input should be a valid string
[Arg] 3 -> input: fmp -> Input should be a valid integer, unable to parse string as an integer

In [102]:
r = llm(query=f"what was the latest PE ratio for stock symbol MRK")
display(Markdown(r.replace("$", "\$")))


The latest PE (Price-to-Earnings) ratio for Merck & Co., Inc. (MRK) is **139.75**.

In [109]:
r = llm(query=f"what was the most recent price to sales ratio for stock symbol {symbol}")
display(Markdown(r.replace("$", "\$")))


OpenBBError: 
[Error] -> 2 validations error(s)
[Arg] 2 -> input: 1 -> Input should be a valid string
[Arg] 3 -> input: fmp -> Input should be a valid integer, unable to parse string as an integer

In [111]:
r = llm(query=f"what was the most recent revenue per share for stock symbol {symbol}")
display(Markdown(r.replace("$", "\$")))


OpenBBError: 
[Error] -> 1 validations error(s)
[Arg] 2 -> input: fmp -> Input should be 'annual' or 'quarter'

In [117]:
r = llm(query=f"what was the latest dividend data for stock symbol {symbol}")
display(Markdown(r.replace("$", "\$")))


OpenBBError: 
[Error] -> 1 validations error(s)
[Arg] start_date -> input: nasdaq -> Input should be a valid date or datetime, input is too short

In [116]:
r = llm(query=f"what was the dividend data at the end of 2023 for stock symbol {symbol}")
display(Markdown(r.replace("$", "\$")))


At the end of 2023, the dividend data for Merck & Co., Inc. (MRK) was as follows:

- **Ex-Dividend Date:** March 14, 2024
- **Amount:** \$0.77 per share
- **Adjusted Dividend:** \$0.77 per share
- **Record Date:** March 15, 2024
- **Payment Date:** April 5, 2024
- **Declaration Date:** January 23, 2024

This implies that the dividend was declared at the beginning of 2024 and scheduled for payment in April 2024.

In [119]:
r = llm(query=f"what was the dividend yield for stock symbol {symbol} as of the end of 2023?")
display(Markdown(r.replace("$", "\$")))


The trailing dividend yield for the stock symbol MRK (Merck & Co., Inc.) as of the end of 2023 was:

- **December 29, 2023:** 2.715%

Is there anything else you would like to know?

In [123]:
r = llm(query=f"what was the performance for symbol MRK from 1 year ago?")
display(Markdown(r.replace("$", "\$")))


To evaluate the performance of Merck & Co., Inc. (MRK) over the past year, I would typically look at stock price data, including the opening price from one year ago and the current price, to compute the performance. However, I currently have data on the equity exposure of MRK across various ETFs, indicating its significant presence in multiple funds.

If you're seeking stock performance information, I recommend checking stock charts or financial news websites for MRK.

Would you like me to provide help with a specific aspect of the MRK stock data, or should I assist you with something else?

In [125]:
r = llm(query=f"which ETF has the highest weight in symbol MRK?")
display(Markdown(r.replace("$", "\$")))


The ETF with the highest weight in MRK (Merck & Co., Inc.) is:

**XLV - Health Care Select Sector SPDR Fund** with a weight of 6.01%.